In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
from stellargraph import StellarGraph
from stellargraph import datasets
from IPython.display import display, HTML


import pandas as pd
import numpy as np
from stellargraph import StellarGraph
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
import json

In [ ]:
filehandler = open("../data/sample_data/hotel_reservation_sample.pkl","rb")
dat = pickle.load(filehandler)
filehandler.close()
print(type(dat))
print(dat.shape)

In [ ]:
df = dat.sample(frac=1)
df_without_labels = df.copy()
df_without_labels = df_without_labels.drop(columns=['label'])
df.columns
df_check = df[df.label == -1]

In [ ]:
df['has_bottleneck'] = 1
df.loc[df.label == -1,'has_bottleneck'] = 0
df_graph_class = df.copy()
df_graph_class['has_bottleneck'] = 1
df_graph_class.loc[df_graph_class.label == -1,'has_bottleneck'] = 0
df_graph_class = df_graph_class.drop(columns = ['label'])

In [ ]:
graph_edges = pd.DataFrame(
    {"source": ["0", "0", "1", "1", "4"], 
     "target": ["1", "4", "2", "3", "5"]}
)
graph_edges

In [ ]:
node_indices = [str(i) for i in range(len(df_without_labels.columns))]

In [ ]:
df_sample = df.groupby('has_bottleneck', group_keys=False).apply(lambda x: x.sample(frac=0.1))

In [ ]:
df_sample['has_bottleneck'].value_counts()

In [ ]:
graphs = []
graph_labels = []
count = 0
for index,row in df_sample.iterrows():
    count = count + 1
    label = row['has_bottleneck']
    node_features = row.drop(labels = ['label', 'has_bottleneck'])
    graph_nodes = pd.DataFrame({"x": node_features.tolist()}, index = node_indices)
    graph = StellarGraph(graph_nodes, graph_edges)
    graphs.append(graph)
    graph_labels.append(label)
    if count % 10000 == 0:
        print(count)

In [ ]:
graph_label_lists = graph_labels.copy()
graph_labels = pd.Series(graph_label_lists)

In [ ]:
graph_labels.value_counts().to_frame()

In [ ]:
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

In [ ]:
generator = PaddedGraphGenerator(graphs=graphs)

In [ ]:
k = 35  # the number of rows for the output tensor
layer_sizes = [32, 32, 32, 1]

dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_sizes,
    activations=["tanh", "tanh", "tanh", "tanh"],
    k=k,
    bias=False,
    generator=generator,
)
x_inp, x_out = dgcnn_model.in_out_tensors()

In [ ]:
x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)

x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)

x_out = Flatten()(x_out)

x_out = Dense(units=128, activation="relu")(x_out)
x_out = Dropout(rate=0.5)(x_out)

predictions = Dense(units=1, activation="sigmoid")(x_out)

In [ ]:
model = Model(inputs=x_inp, outputs=predictions)

model.compile(
    optimizer=Adam(lr=0.0001), loss=binary_crossentropy, metrics=["acc"],
)

In [ ]:
train_graphs, test_graphs = model_selection.train_test_split(
    graph_labels, train_size=0.9, test_size=None, stratify=graph_labels,
)

In [ ]:
gen = PaddedGraphGenerator(graphs=graphs)

train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    batch_size=50,
    symmetric_normalization=False,
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    batch_size=1,
    symmetric_normalization=False,
)

In [ ]:
epochs = 10

In [ ]:
history = model.fit(
    train_gen, epochs=epochs, verbose=1, validation_data=test_gen, shuffle=True,
)

In [ ]:
sg.utils.plot_history(history)

In [ ]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))